##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gaussian Process Regression in TensorFlow Probability 


In this tutorial, we explore Gaussian process regression using
TensorFlow 2.0 and TensorFlow Probability. We generate some noisy observations from
some known functions and fit GP models to those data. We will then sample from the GP
posterior and plot the sampled function values over grids in their domains. This tutorial will also cover Markov Chain Monte Carlo methods and Deep Gaussian Processes (if time permits). 



## Background

###What is a Gaussian Process?###

<img src="https://www.researchgate.net/profile/Miguel_Lazaro-Gredilla/publication/260637079/figure/fig1/AS:668972609458178@1536506907350/Example-of-a-Gaussian-process-posterior-in-12-with-20-training-samples-denoted-by.png" width="300">

Gaussian processes are distributions over functions $f(x)$, which are defined by a mean function $m(x)$ and positive-definite covariance kernel $k(x, x^{'})$, with x being the function values and $(x, x^{'})$ being all the possible pairs of input data points. A function $f(x)$ of a Gaussian Process is therefore defined as:

$$f(x) \sim \mathcal{GP}(m(x),k(x,x'))$$

where for any finite subset $X = \{x_{1} ... x_{n}\}$ of the domain of $x$, the marginal distribution (probability distribution of the variables contained in the subset), is a multivariate Gaussian distribution, with a mean vector $\mu = m(X)$ and covariance matrix $\Sigma = k(X, X)$. 

## Imports

In [0]:
!pip install tensorflow==2.0.0
!pip install tfp-nightly
!pip install gast==0.2.2
!pip install quandl

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import positive_semidefinite_kernels as tfk
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy
from time import time

print(tf.__version__)
print(tfp.__version__)

## Example: Exact GP Regression on Noisy Sinusoidal Data
Here we generate training data from a noisy sinusoid, then sample a bunch of
curves from the posterior of the GP regression model. We use
[Adam](https://arxiv.org/abs/1412.6980) to optimize the kernel hyperparameters
(we minimize the negative log likelihood of the data under the prior). We
plot the training curve, followed by the true function and the posterior
samples.

In [0]:
def sinusoid(x):
  return np.sin(3 * np.pi * x[..., 0])

def generate_1d_data(num_training_points, observation_noise_variance):
  """Generate noisy sinusoidal observations at a random set of points.

  Returns:
     observation_index_points, observations
  """
  index_points_ = np.random.uniform(-1., 1., (num_training_points, 1))
  index_points_ = index_points_.astype(np.float64)

  observations_ = (sinusoid(index_points_) +
                   np.random.normal(loc=0,
                                    scale=np.sqrt(observation_noise_variance),
                                    size=(num_training_points)))
  return index_points_, observations_

In [0]:
# Generate training data with a known noise level (we'll later try to recover
# this value from the data).
NUM_TRAINING_POINTS = 100
observation_index_points_, observations_ = generate_1d_data(
    num_training_points=NUM_TRAINING_POINTS,
    observation_noise_variance=.1)

#Let's visualise our training data 
index_points_ = np.linspace(-1.2, 1.2, 200, dtype=np.float64)
index_points_ = index_points_[..., np.newaxis]

plt.figure(figsize=(12, 4))
plt.title('Example Sinusoid Data')
plt.plot(index_points_, sinusoid(index_points_),
         label='True fn')
plt.scatter(observation_index_points_[:, 0], observations_,
            label='Observations')
plt.legend()
plt.show()

###Covariance function as a prior###


<img src= "http://inverseprobability.com/talks/slides/diagrams/kern/eq_covariance.gif">

The covariance function (a.k.a kernel function), which describes the covariance between each input pair, is what determines the distribution over a sample function $f(x)$. Therefore, we are effectively setting a prior information over our data by choosing a specific covariance function. 

In this tutorial, we make use of the
ExponentiatedQuadratic covariance function (though you're more than welcome to try other covariance functions from the TFP library). Its form is

$$
k(x, x') := \sigma^2 \exp \left( \frac{\|x - x'\|^2}{\lambda^2} \right)
$$

where $\sigma^2$ is the *overall variance* ($\sigma$ is sometimes called the 'amplitude') and $\lambda$ the *length scale*.
We will now define these kernel parameters and optimise them via a maximum likelihood procedure.

The hyperparameter *observation noise variance* is an extra hyperparameter to be learned by our Gaussian Process model [(McHutchon & Rasmussen 2011)](http://mlg.eng.cam.ac.uk/pub/pdf/MchRas11.pdf). It allows us to take the noise in the observation into account. 

In [0]:
# Create the trainable model parameters, which we'll subsequently optimize.
# Note that we constrain them to be strictly positive.
amplitude_ = tf.Variable(initial_value=1., name='amplitude_', dtype=np.float64)
length_scale_ = tf.Variable(initial_value=1., name='length_scale_', dtype=np.float64)
observation_noise_variance_ = tf.Variable(initial_value=1e-6,
                                         name='observation_noise_variance_',
                                         dtype=np.float64)

In [0]:
# Illustrate covariance matrix and function

#define the covariance function 
def exponentiated_quadratic(xa, xb):
    #Exponentiated quadratic  with σ=1
    # L2 distance (Squared Euclidian)
    return np.exp(-0.5 * scipy.spatial.distance.cdist(xa, xb, 'sqeuclidean'))
    
# Show covariance matrix from exponentiated quadratic function
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
xlim = (-3, 3)
X = np.expand_dims(np.linspace(*xlim,num=25), 1)
sigma = exponentiated_quadratic(X, X)

# Plot covariance matrix
im = ax1.imshow(sigma, cmap=cm.YlGnBu)
cbar = plt.colorbar(
    im, ax=ax1, fraction=0.045, pad=0.05)
cbar.ax.set_ylabel('$k(x,x)$', fontsize=10)
ax1.set_title((
    'Exponentiated quadratic \n'
    'example of covariance matrix'))
ax1.set_xlabel('x', fontsize=13)
ax1.set_ylabel('x', fontsize=13)
ticks = list(range(-3, 4))
ax1.set_xticks(np.linspace(0, len(X)-1, len(ticks)))
ax1.set_yticks(np.linspace(0, len(X)-1, len(ticks)))
ax1.set_xticklabels(ticks)
ax1.set_yticklabels(ticks)
ax1.grid(False)

# Show covariance with X=0
xlim = (-4, 4)
X = np.expand_dims(np.linspace(*xlim, num=100), 1)
zero = np.array([[0]])
sigma0 = exponentiated_quadratic(X, zero)

# Make the plots
ax2.plot(X[:,0], sigma0[:,0], label='$k(x,0)$')
ax2.set_xlabel('x', fontsize=13)
ax2.set_ylabel('covariance', fontsize=13)
ax2.set_title((
    'Exponentiated quadratic  covariance\n'
    'between $x$ and $0$'))
ax2.set_xlim(*xlim)
ax2.legend(loc=1)

fig.tight_layout()
plt.show()

###Optimising the Hyperparameters###

Let's define the marginal likelihood $p(y | X, \theta)$ of the GP distribution. By maximizing it based on our observed data $(X, y)$, we can find the optimal hyperparameters $\hat\theta$ of our GP model. The optimal hyperparameter is defined as follows:

$$\hat\theta = \underset{\theta}{\arg\max}(p(y | X, \theta))$$

The marginal likelihood of the GP is the likelihood of a multivariate Gaussian distribution which is given as:

$$p(y | \mu, \Sigma) = \frac{1}{\sqrt{(2\pi)^d|\Sigma|}} \exp(-\frac{1}{2}(y-\mu)^{\top}\Sigma^{-1}(y-\mu)) $$

In our case, $\mu$ and $\Sigma$ are defined by the mean function $m_{\theta}(x)$ and the covariance (kernel) function $k_{\theta}(x,x')$ of our GP model. Therefore, the likelihood could be re-written as:

$$p(y | \mu, \theta) = \frac{1}{\sqrt{(2\pi)^d|\Sigma_{\theta}|}} \exp(-\frac{1}{2}(y-\mu_{\theta})^{\top}\Sigma_{\theta}^{-1}(y-\mu_{\theta})) $$

where $d$ is the dimensionality of the marginal and $|\Sigma_{\theta}|$ is the determinant of the covariance kernel. Now let's get rid of the exponent on the right hand side by taking the log on both sides. 

$$\log p(y | X, \theta) = -\frac{1}{2}(y - \mu_{\theta})^{\top} \Sigma_{\theta}^{-1} (y - \mu_{\theta}) - \frac{1}{2} \log |\Sigma_{\theta}| - \frac{d}{2}\log 2\pi$$

The optimal hyperparameter $\hat\theta$ can therefore be found by minimizing the negative log marginal likelihood:

$$\hat\theta = \underset{\theta}{\arg\max}(p(y | X, \theta)) = \underset{\theta}{\arg\min}\log p(y | X, \theta)$$

*Note that the function neg_log_likelihood is decorated with @tf.function. This allows us to compile our function into a high performance Tensorflow graph, giving us the benefit of faster execution while still using natural Python syntax.*

In [0]:
@tf.function #  <- faster execution 
def neg_log_likelihood():
    amplitude = np.finfo(np.float64).tiny + tf.nn.softplus(amplitude_)
    length_scale = np.finfo(np.float64).tiny + tf.nn.softplus(length_scale_)
    observation_noise_variance = np.finfo(np.float64).tiny + tf.nn.softplus(observation_noise_variance_)

    kernel = tfk.ExponentiatedQuadratic(amplitude, length_scale)

    gp = tfd.GaussianProcess(
        kernel=kernel,
        index_points=observation_index_points_,
        observation_noise_variance=observation_noise_variance
    )

    return -gp.log_prob(observations_)

The [Adam optimizer](https://arxiv.org/pdf/1412.6980.pdf) will be used to tune our hyperparameters.



In [0]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)

One of the key benefits of TF2.0 over TF1.0 is the use of eager execution by default. Eager Execution means tf.Tensor will work seamlessly with NumPy arrays and standard Python debugging tools can be used for immediate error reporting and inspection of results. 

Training and/or gradient computation in eager mode is done using tf.GradientTape.

In [0]:
# Now we optimize the model parameters.
num_iters = 1000

#initialize negative log-likelihood as an array so that we can print out the training history for inspection later on 
nlls = np.zeros(num_iters, np.float64)

start = time()
for i in range(num_iters):
    nlls[i] = neg_log_likelihood()
    with tf.GradientTape() as tape:
        loss = neg_log_likelihood()
    grads = tape.gradient(loss, [amplitude_, length_scale_, observation_noise_variance_])
    optimizer.apply_gradients(zip(grads, [amplitude_, length_scale_, observation_noise_variance_]))

print("Time: %0.2f" % (time() - start))
print('Trained parameters:'.format(amplitude_))
print('amplitude: {}'.format(amplitude_))
print('length_scale: {}'.format(length_scale_))
print('observation_noise_variance: {}'.format(observation_noise_variance_))

In [0]:
# Plot the loss evolution
plt.figure(figsize=(12, 4))
plt.plot(nlls)
plt.title('Negative Log-Marginal Likelihood during training')
plt.xlabel("Training iteration")
plt.ylabel("NLL")
plt.show()

In [0]:
#let's re-define the kernel using the trained parameters
amplitude = np.finfo(np.float64).tiny + tf.nn.softplus(amplitude_)
length_scale = np.finfo(np.float64).tiny + tf.nn.softplus(length_scale_)
observation_noise_variance = np.finfo(np.float64).tiny + tf.nn.softplus(observation_noise_variance_)

kernel = tfk.ExponentiatedQuadratic(amplitude, length_scale)

Using the trained hyperparameters, let's sample from the posterior distribution $p(y| X, \theta)$ conditioned on observations $X$ and hyperparameters $\theta$. We will draw samples at points other than the training inputs.

In [0]:
predictive_index_points_ = np.linspace(-1.2, 1.2, 200, dtype=np.float64)
# Reshape to [200, 1] -- 1 is the dimensionality of the feature space.
predictive_index_points_ = predictive_index_points_[..., np.newaxis]

gprm = tfd.GaussianProcessRegressionModel(
    kernel=kernel,  # Reuse the same kernel instance, with the same params
    index_points=predictive_index_points_,
    observation_index_points=observation_index_points_,
    observations=observations_,
    observation_noise_variance=observation_noise_variance,
    predictive_noise_variance=0.)

#compute the mean and standard deviation of the predictions from the trained model
posterior_mean = gprm.mean()
posterior_std = gprm.stddev()

In [0]:
# Plot the true function, observations, and posterior samples.
plt.figure(figsize=(16, 8))
plt.plot(predictive_index_points_, sinusoid(predictive_index_points_),
         label='True fn')
plt.scatter(observation_index_points_[:, 0], observations_,
            label='Observations')

plt.plot(predictive_index_points_, posterior_mean, c='r', label='Posterior Mean $\mu$')
plt.fill_between(predictive_index_points_[:,0],  posterior_mean - 2*posterior_std, posterior_mean + 2*posterior_std, alpha=.5, label='$\mu \pm 2\sigma$')

leg = plt.legend(loc='upper right')
for lh in leg.legendHandles: 
    lh.set_alpha(1)
plt.xlabel(r"Index points ($\mathbb{R}^1$)")
plt.ylabel("Observation space")
plt.show()

*Note: if you run the above code several times, sometimes it looks great and
other times it looks terrible! The maximum likelihood training of the parameters
is quite sensitive and sometimes converges to poor models. The best approach
is to use MCMC to marginalize the model hyperparameters.*

## Marginalisation of model hyperparameters using Markov-Chain Monte Carlo

Here we use Tensorflow Probability's MCMC functionality to marginalise GP hyperparameters (kernel parameters & observation noise variance). Markov-Chain Monte Carlo (MCMC) techniques are methods for sampling from probability distributions using Markov chains. 

### What is Markov-Chain Monte Carlo?  ###
To first understand what Monte Carlo is, let's take a distribution $p(x)$ and compute its expectation $E[x]$. The expectation is given as:

$$ E[x] = \int_{-\infty}^{\infty} xp(x) dx $$ 

Monte Carlo method simply approximates this integral by drawning $n$ samples from $p(x)$ and then evaluating:

$$ E[x] \approx \frac{1}{n}\sum_{i = 1}^{\infty} x_{i}$$

Similarly, predictive distribution for our GP model is computed by:

$$ p(y^{*}|x^{*}, x, y) = \int p(y^{*}|x^{*}, x, y, \theta) p(\theta|y,x)d\theta \approx \frac{1}{N}\sum_{n = 1}^{N} p(y^{*}|x^{*}, x, y, \theta^{n}), \quad\theta^{n} \sim p(\theta|x,y)$$

The issue here is that the integral is analytically intractable to compute as we cannot easily sample from $p(\theta|x,y)$. To sample $p(\theta|x,y)$, we need something else and that's where the Markov chain comes in. 

Consider a sequence of random variables $\{x_{0}, x_{1} ..., x_{n}\}$ sampled from a probability distribution $p(x_{t+1} | x_{t})$. A Markov chain is a sequence in which the probability of each next sample $x_{t+1}$ only depends on the current state $x_{t}$ and not on the further history $\{x_{0}, x_{1}, .., x_{t-1}\}$. 

Thus, the MCMC techniques attempts to approximate a target distribution by constructing cleverly sampled chains that draw samples, which are progressively more likely realisations of the distribution of interest; in the context of GP, that would be our hyperparameter posterior distribution $p(\theta|y,x)$. 

Let's first define the log posterior distribution $\log p(\theta|y,x)$ and initialise our Markov chain.

In [0]:
def joint_log_prob(
    index_points, observations, amplitude, length_scale, noise_variance):

  # Hyperparameter Distributions.
  rv_amplitude = tfd.LogNormal(np.float64(0.), np.float64(1))
  rv_length_scale = tfd.LogNormal(np.float64(0.), np.float64(1))
  rv_noise_variance = tfd.LogNormal(np.float64(0.), np.float64(1))

  gp = tfd.GaussianProcess(
      kernel=tfk.ExponentiatedQuadratic(amplitude, length_scale),
      index_points=index_points,
      observation_noise_variance=noise_variance)

  return (
      rv_amplitude.log_prob(amplitude) +
      rv_length_scale.log_prob(length_scale) +
      rv_noise_variance.log_prob(noise_variance) +
      gp.log_prob(observations)
  )

# Construct a Markov chain of hyperparameter variables that we want to optimise
initial_chain_states = [
    1e-1 * tf.ones([], dtype=np.float64, name='init_amplitude'),
    1e-1 * tf.ones([], dtype=np.float64, name='init_length_scale'),
    1e-1 * tf.ones([], dtype=np.float64, name='init_obs_noise_variance')
]

#log_posterior distribution of the hyperparameters 
def unnormalized_log_posterior(amplitude, length_scale, noise_variance):
  return joint_log_prob(
      observation_index_points_, observations_, amplitude, length_scale,
      noise_variance)

*Note: Inference in MCMC is usually performed while ignoring the normalizing constant $p(y)$. Therefore, $p(\theta| y) \propto p(y|\theta)p(\theta)$ and the log hyperparameter posterior distribution can be expressed as $\log p(y|\theta) + \log p(\theta)$. *

###Metropolis-Hastings Algorithm###

Now that we've initialised our Markov chain and defined our posterior hyperparameter distribution, we will use the **Metropolis-Hastings algorithm** to converge to our target distribution. 

The following things are to be specified for the Markov chain:

*   probability distribution for the initial state $p(\theta^{1})$
*   Conditional probability for subsequent states in the form of transition probabilities $T(\theta^{t+1} \leftarrow \theta^{t})$

Note that $T(\theta^{t+1} \leftarrow \theta^{t})$ is also known as the *transition kernel*. 

In the case of Metropolis-Hastings algorithm, the Markov chain transition operator is defined as follows:

*   A new candidate state $\theta^{*}$ proposed by a proposal distribution $q(\theta^{*}|\theta)$
*   The probability of accepting the candidate state is $\min(1, r)$, where $r = \frac{p(\theta^{*}|y)q(\theta^{*}|\theta^{t})}{p(\theta^{t}|y)q(\theta^{t}|\theta^{*})}$ 

If the proposal is accepted, set $\theta^{t+1} = \theta^{*}$, otherwise $\theta^{t+1} = \theta^{t}$. 

###Hamiltonian Monte Carlo###

**HMC (Hamiltonian Monte Carlo)** is a special case of Metropolis-Hastings algorithm that uses Hamiltonian dynamics to create proposal distribution for the Metropolis-Hastings algorithm that allows the state space to be explored more efficiently. The mathematical details of the algorithm is beyond the scope of this tutorial so we will only cover a brief summary of the algorithm, which is as follows:

For a given number of iterations,

1.   Sample a new momentum value $\rho$ from an auxillary distribution.
2.   Evolve the joint system $(\theta, \rho)$ via Hamilton's equations.
3. Update $\theta$ using the leapfrog integrator with discretisation time $\epsilon$ and leapfrog steps L according to the Hamiltonian dynamics.
4. Apply the Metropolis acceptance step to decide whether to update to a new state $(\theta^{*}, \rho^{*})$ or keep the existing state. 

For mathematical details, please refer to [Betancourt & Girolami 2013](https://arxiv.org/pdf/1312.0906.pdf) and [Neal 2012](https://arxiv.org/pdf/1206.1901.pdf). 


In [0]:
##Transport map accelerated Markov chain Monte Carlo: https://arxiv.org/pdf/1412.5492.pdf##
# Since HMC operates over unconstrained space, we need to transform the
# samples so they live in real-space.
unconstraining_bijectors = [
    tfp.bijectors.Softplus(),
    tfp.bijectors.Softplus(),
    tfp.bijectors.Softplus(),
]

num_results = 200

@tf.function   # <- make things fast
def run_mcmc():
  return tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=500,
    num_steps_between_results=3,
    current_state=initial_chain_states,
    kernel=tfp.mcmc.TransformedTransitionKernel(
        inner_kernel = tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=unnormalized_log_posterior,
            step_size=[np.float64(.15)],
            num_leapfrog_steps=3),
        bijector=unconstraining_bijectors))
  
[amplitudes,length_scales,observation_noise_variances], kernel_results = run_mcmc()

print("Acceptance rate: {}".format(
    np.mean(kernel_results.inner_results.is_accepted)))

In [0]:
gprm = tfd.GaussianProcessRegressionModel(
    kernel=tfk.ExponentiatedQuadratic(amplitudes, length_scales),
    index_points=predictive_index_points_,
    observation_index_points=observation_index_points_,
    observations=observations_,
    observation_noise_variance=observation_noise_variances,
    predictive_noise_variance=0.)

samples = np.transpose(gprm.sample())

posterior_mean = np.mean(samples, axis=1)
posterior_std = np.std(samples, axis=1)

# Plot the true function, observations, and posterior samples.
plt.figure(figsize=(16, 8))
plt.plot(predictive_index_points_, sinusoid(predictive_index_points_),
         label='True fn')
plt.scatter(observation_index_points_[:, 0], observations_,
            label='Observations')

plt.plot(predictive_index_points_, posterior_mean, c='r', label='Posterior Prediction')
plt.fill_between(predictive_index_points_[:,0],  posterior_mean - 2*posterior_std, posterior_mean + 2*posterior_std, alpha=.5)

leg = plt.legend(loc='upper right')
for lh in leg.legendHandles: 
    lh.set_alpha(1)
plt.xlabel(r"Index points ($\mathbb{R}^1$)")
plt.ylabel("Observation space")
plt.show()